In [1]:
import csv
import codecs
import urllib.request
import urllib.error
import sys
import pandas as pd

1) Creamos variables para los parámetros de la consulta

In [2]:
BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

ApiKey='8CG3ETAC2GNVGSG2G72ZAS3EQ' # Coloca acá tu clave de Visual Crossing
#UnitGroup sets the units of the output - us or metric
UnitGroup='us'

#Location for the weather data
Location='40.776676,-73.971321'

#Optional start and end dates
#If nothing is specified, the forecast is retrieved. 
#If start date only is specified, a single historical or forecast day will be retrieved
#If both start and and end date are specified, a date range will be retrieved

StartDate = '2018-01-01' #Hasta el momento sólo descargué los datos del 01 al 10 de Enero para esta locación. Puede comenzar a descargar a partir del 11 de Enero
EndDate='2018-01-31'

#JSON or CSV 
#JSON format supports daily, hourly, current conditions, weather alerts and events in a single JSON package
#CSV format requires an 'include' parameter below to indicate which table section is required
ContentType="csv"

#include sections
#values include days,hours,current,alerts
Include="days"

In [3]:
#basic query including location
ApiQuery=BaseURL + Location

#append the start and end date if present
if (len(StartDate)):
    ApiQuery+="/"+StartDate
    if (len(EndDate)):
        ApiQuery+="/"+EndDate

#Url is completed. Now add query parameters (could be passed as GET or POST)
ApiQuery+="?"

#append each parameter as necessary
if (len(UnitGroup)):
    ApiQuery+="&unitGroup="+UnitGroup

if (len(ContentType)):
    ApiQuery+="&contentType="+ContentType

if (len(Include)):
    ApiQuery+="&include="+Include

ApiQuery+="&key="+ApiKey

2) Se realiza la consulta y se guarda en la variable CSVBytes

In [4]:

print(' - Running query URL: ', ApiQuery)
print()

try: 
    CSVBytes = urllib.request.urlopen(ApiQuery)
except urllib.error.HTTPError  as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code, ErrorInfo)
    sys.exit()
except  urllib.error.URLError as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code,ErrorInfo)
    sys.exit()

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/40.776676,-73.971321/2018-01-01/2018-01-31?&unitGroup=us&contentType=csv&include=days&key=8CG3ETAC2GNVGSG2G72ZAS3EQ



3) Se parsea CSVBytes en un objeto de tipo csv.reader

In [5]:
# Parse the results as CSV
CSVText = csv.reader(codecs.iterdecode(CSVBytes, 'utf-8'))

4) Se itera sobre CSVText donde cada iteración arroja una lista de strings. Cada lista es una fila de la tabla, así que guadaremos cada fila en otra lista (creando una lista de listas)

In [6]:
lista = []
for row in CSVText:
    lista.append(row)

5) Con la lista de listas, se crea un data frame. El primer objeto de la lista serán las columnas, el resto es el contenido

In [7]:
df = pd.DataFrame(lista[1:],columns=lista[0])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              31 non-null     object
 1   datetime          31 non-null     object
 2   tempmax           31 non-null     object
 3   tempmin           31 non-null     object
 4   temp              31 non-null     object
 5   feelslikemax      31 non-null     object
 6   feelslikemin      31 non-null     object
 7   feelslike         31 non-null     object
 8   dew               31 non-null     object
 9   humidity          31 non-null     object
 10  precip            31 non-null     object
 11  precipprob        31 non-null     object
 12  precipcover       31 non-null     object
 13  preciptype        31 non-null     object
 14  snow              31 non-null     object
 15  snowdepth         31 non-null     object
 16  windgust          31 non-null     object
 17  windspeed         

In [17]:
df[['datetime','snowdepth']]

,datetime,snowdepth
0,2018-01-01,0
1,2018-01-02,0
2,2018-01-03,0.2
3,2018-01-04,2.3
4,2018-01-05,6.5
5,2018-01-06,6
6,2018-01-07,5.7
7,2018-01-08,5.2
8,2018-01-09,4.9
9,2018-01-10,3.8


6) Guardo el resultado de la consulta como un archivo csv

In [18]:
df.to_csv('weather_data_02.csv',index=False)